In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [24]:
!ls sample_data/


anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving rt-polarity.neg to rt-polarity.neg
Saving rt-polarity.pos to rt-polarity.pos
User uploaded file "rt-polarity.neg" with length 612290 bytes
User uploaded file "rt-polarity.pos" with length 626167 bytes


In [0]:
import nltk
posFilePath='rt-polarity.pos'
negFilePath='rt-polarity.neg'

In [0]:
with open(posFilePath, encoding="latin-1") as f:
    posData=f.readlines()
    
with open(negFilePath,encoding="latin-1") as f:
    negData=f.readlines()
    

In [0]:
testSpltIndx=2500
testNegRev=negData[testSpltIndx+1:]
testPosRev=posData[testSpltIndx+1:]

trainNegRev=negData[:testSpltIndx]
trainPosRev=posData[:testSpltIndx]

In [0]:
def getVocab():
    posWordLst = [word for line in trainPosRev for word in line.split()]
    negWordLst = [word for line in trainNegRev for word in line.split()]
    allWordsList = posWordLst+negWordLst
    vocab = list(set(allWordsList))
    
    return vocab


In [0]:
def getTrainingData():
    negTaggedRevLst = [{'review':review.split(),'label':'negative'} for review in trainNegRev]
    posTaggedRevLst = [{'review':review.split(),'label':'positive'} for review in trainPosRev]
    fullyTaggedTrainData = negTaggedRevLst+posTaggedRevLst

    trainData =[(reviewObj['review'],reviewObj['label']) for reviewObj in           fullyTaggedTrainData]
    return trainData


In [0]:
def extractFeatures(review):
    review_words=set(review)
    features={}
    for word in vocabulary:
        features[word]=(word in review_words)
    
    return features


In [0]:
def getTrainedNaiveBayesClassifier(extract_feaures,trainingData):
    trainingFeatures = nltk.classify.apply_features(extract_feaures,trainingData)
    trainedNBClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)
    return trainedNBClassifier    


In [0]:
vocabulary = getVocab()
trainingData = getTrainingData()

#trainedNBClassifier = getTrainedNaiveBayesClassifier(extractFeatures,trainingData)


In [19]:
def nbSentimentCalc(review):
    problemInstance = review.split()
    problemFeatures = extractFeatures(problemInstance)
    return trainedNBClassifier.classify(problemFeatures)

#nbSentimentCalc("what an bad good movie")
def getTestReviewSetiment(nbSentimentCalc):
    testNegRes = [nbSentimentCalc(review) for review in testNegRev]
    testPosRes = [nbSentimentCalc(review) for review in testPosRev]
    
    labelToNum = {'positive':1,'negative':-1}
    numericNegRes = [labelToNum[x] for x in testNegRes]
    numericPosRes = [labelToNum[x] for x in testPosRes]
    return {'results-on-positive':numericPosRes,'results-on-negetive':numericNegRes}

def runDiagnostics(reviewResult):
    posReviewRes = reviewResult['results-on-positive']
    negReviewRes = reviewResult['results-on-negetive']
    
    numTruePositive = sum(x>0 for x in posReviewRes)
    numTreuNegetive = sum(x<0 for x in negReviewRes)
    
    pctTruePos = float(numTruePositive)/len(posReviewRes)
    pctTrueNeg = float(numTreuNegetive)/len(negReviewRes)
    totalAccurate = numTreuNegetive+numTruePositive
    total = len(posReviewRes)+len(negReviewRes)
    print("accuracy of +ve rev ="+"%.2f"%(pctTruePos*100)+"%")
    print("Accuracy of -ve rev ="+"%.2f"%(pctTrueNeg*100)+"%")
    
runDiagnostics(getTestReviewSetiment(nbSentimentCalc))


accuracy of +ve rev =73.43%
Accuracy of -ve rev =76.96%
